# Google Colab - Gemini 视频描述生成工作流

这个 Notebook 用于在 Google Colab 中使用 Gemini API 生成视频描述。

**优点:**
- ✓ 无 VPN 地理限制（云端执行）
- ✓ 免费计算资源
- ✓ Google Drive 集成
- ✓ 自动保存和下载结果

# Google Colab - Gemini 视频描述生成

这个 Notebook 用于在 Google Colab 中运行 Gemini API，生成视频描述。

**优点:**
- ✓ 无 VPN 地理限制
- ✓ 免费计算资源
- ✓ Google Drive 集成
- ✓ 自动保存和下载

## Step 1: 安装依赖

In [ ]:
!pip install google-generativeai pandas numpy -q
print("✓ 依赖安装完成")

## Step 2: 导入库和设置 API

In [ ]:
import google.generativeai as genai
import pandas as pd
import json
from datetime import datetime
from google.colab import drive, files
import time
from tqdm import tqdm

GEMINI_API_KEY = "your_api_key_here"
genai.configure(api_key=GEMINI_API_KEY)

print("✓ API 已配置")

## Step 3: 挂载 Google Drive

In [ ]:
drive.mount('/content/drive')
csv_path = '/content/drive/MyDrive/QA_pair_v1_3options.csv'
print(f"✓ Drive 已挂载")

## Step 4: 加载数据

In [ ]:
def load_qa_data(csv_path):
    df = pd.read_csv(csv_path)
    data = []
    for idx, row in df.iterrows():
        facts = []
        for i in range(1, 7):
            q_col = f'q{i}_text'
            a_col = f'q{i}_ans_correct'
            if q_col in df.columns and a_col in df.columns:
                q_text = row.get(q_col, '')
                q_ans = row.get(a_col, '')
                if pd.notna(q_text) and pd.notna(q_ans):
                    q_text = str(q_text).strip().replace('\n', ' ')
                    q_ans = str(q_ans).strip().replace('\n', ' ')
                    facts.append(f"Fact {i}: {q_text} Answer: {q_ans}.")
        data.append({'video_id': str(row['video_number']), 'facts_text': '\n'.join(facts)})
    return data

data = load_qa_data(csv_path)
print(f"✓ 加载了 {len(data)} 条数据")

## Step 5: 生成描述

In [ ]:
def generate_description(video_id, facts_text):
    try:
        prompt = f"Based on these facts:\n\n{facts_text}\n\nGenerate a detailed description (300-400 words)."
        model = genai.GenerativeModel("models/gemini-2.0-flash-exp")
        response = model.generate_content(prompt)
        if response and response.text:
            return {'video_id': video_id, 'status': 'success', 'description': response.text}
        else:
            return {'video_id': video_id, 'status': 'failed', 'error': 'Empty response'}
    except Exception as e:
        return {'video_id': video_id, 'status': 'failed', 'error': str(e)}

print("✓ 函数已定义")

## Step 6: 批量处理

In [ ]:
def process_batch(data, num_items=10):
    results = []
    for i, item in enumerate(tqdm(data[:num_items])):
        result = generate_description(item['video_id'], item['facts_text'])
        results.append(result)
        time.sleep(1)
    return results

results = process_batch(data, num_items=10)
success = sum(1 for r in results if r['status'] == 'success')
print(f"✓ 完成! 成功: {success}/{len(results)}")

## Step 7: 保存结果

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f'results_{timestamp}.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
files.download(output_file)
print(f"✓ 结果已下载: {output_file}")

# 🚀 Gemini 视频描述生成 - Colab 版本

在 Google Colab 运行，避免本地网络限制

## 📋 使用步骤
1. 设置 Gemini API 密钥
2. 上传或挂载数据文件
3. 运行推理工作流
4. 下载结果

## 步骤 1: 安装依赖

In [ ]:
# 安装必要的包
!pip install google-generativeai pandas numpy -q
print("✓ 依赖安装完成")

## 步骤 2: 设置 Gemini API 密钥

⚠️ **替换为你的实际密钥**
- 获取地址: https://aistudio.google.com/apikey

In [ ]:
import google.generativeai as genai
import os

# ⚠️ 在这里设置你的 API 密钥
GEMINI_API_KEY = "your_gemini_api_key_here"  # 替换为实际密钥

# 配置 API
genai.configure(api_key=GEMINI_API_KEY)

# 验证密钥
try:
    models = list(genai.list_models())
    print(f"✓ API 密钥有效，可用模型: {len(models)} 个")
    print(f"  推荐模型: gemini-2.0-flash")
except Exception as e:
    print(f"❌ API 密钥验证失败: {e}")
    print("请检查密钥是否正确")

## 步骤 3: 数据加载

### 方法 A: 从 Google Drive 挂载（推荐）

In [ ]:
from google.colab import drive
import pandas as pd

# 挂载 Google Drive
drive.mount('/content/drive')

# 设置 CSV 文件路径（根据你的实际路径修改）
# 可以先上传 CSV 到 Google Drive，然后复制路径
csv_path = '/content/drive/MyDrive/QA_pair_v1_3options.csv'

print(f"✓ Google Drive 已挂载")
print(f"CSV 路径: {csv_path}")

### 方法 B: 直接上传文件

In [ ]:
# 如果不用 Drive，可以直接上传文件
from google.colab import files

# 点击上传按钮，选择 QA_pair_v1_3options.csv
uploaded = files.upload()

# 获取上传的文件名
csv_filename = list(uploaded.keys())[0]
csv_path = f'/content/{csv_filename}'

print(f"✓ 文件已上传: {csv_path}")

## 步骤 4: 定义数据加载器

In [ ]:
import pandas as pd
from typing import List, Dict, Any

def format_qa_pair(question: str, answer: str, fact_num: int) -> str:
    """格式化单个 QA 对"""
    question = question.strip().replace('\n', ' ')
    answer = answer.strip().replace('\n', ' ')
    return f"Fact {fact_num}: {question} Answer: {answer}."

def extract_facts_from_row(row: pd.Series) -> str:
    """从数据行提取所有 QA 对"""
    facts = []
    for i in range(1, 7):
        q_text_col = f'q{i}_text'
        q_ans_col = f'q{i}_ans_correct'
        
        if q_text_col in row and q_ans_col in row:
            if pd.notna(row[q_text_col]) and pd.notna(row[q_ans_col]):
                fact_text = format_qa_pair(
                    row[q_text_col], 
                    row[q_ans_col], 
                    i
                )
                facts.append(fact_text)
    
    return '\n'.join(facts)

def load_qa_data(csv_path: str) -> List[Dict[str, Any]]:
    """加载并处理 QA 数据"""
    print(f"📁 正在加载数据: {csv_path}")
    df = pd.read_csv(csv_path)
    print(f"✓ 成功加载 {len(df)} 条记录")
    
    processed_data = []
    for idx, row in df.iterrows():
        video_id = row['video_number']
        facts_text = extract_facts_from_row(row)
        
        processed_data.append({
            'video_id': video_id,
            'facts_text': facts_text
        })
    
    print(f"✓ 处理完成")
    return processed_data

# 测试数据加载
data = load_qa_data(csv_path)
print(f"\n示例数据 (Video {data[0]['video_id']}):")
print(data[0]['facts_text'][:200] + "...")

## 步骤 5: 定义 Gemini 推理函数

In [ ]:
import time
from typing import Optional

def build_prompt(facts_text: str) -> str:
    """构建推理 Prompt"""
    prompt = f"""Based on the following facts about a traffic accident video:

{facts_text}

Please generate a detailed and coherent description of the accident scene, including:
1. Weather and lighting conditions
2. Traffic environment
3. Road configuration
4. Type of collision
5. Primary cause
6. Prevention measures

Keep the description concise and professional (300-400 words)."""
    return prompt

def generate_description(video_id: int, facts_text: str, model_name: str = "gemini-2.0-flash") -> Dict:
    """使用 Gemini 生成描述"""
    try:
        # 构建 Prompt
        prompt = build_prompt(facts_text)
        
        # 创建模型
        model = genai.GenerativeModel(model_name)
        
        # 生成内容
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.7,
                max_output_tokens=512,
            )
        )
        
        if response and hasattr(response, 'text'):
            return {
                'video_id': video_id,
                'status': 'success',
                'description': response.text,
                'facts_text': facts_text,
                'model': model_name
            }
        else:
            return {
                'video_id': video_id,
                'status': 'failed',
                'error': 'Empty response'
            }
            
    except Exception as e:
        return {
            'video_id': video_id,
            'status': 'failed',
            'error': str(e)
        }

# 测试单条推理
print("测试推理...")
test_item = data[0]
test_result = generate_description(test_item['video_id'], test_item['facts_text'])

if test_result['status'] == 'success':
    print(f"\n✓ 推理成功！")
    print(f"\nVideo {test_result['video_id']} 描述:")
    print("=" * 80)
    print(test_result['description'][:300] + "...")
    print("=" * 80)
else:
    print(f"\n❌ 推理失败: {test_result['error']}")

## 步骤 6: 批量处理数据

In [ ]:
import json
from tqdm import tqdm

def process_batch(data: List[Dict], num_items: int = 10, delay: float = 1.0) -> List[Dict]:
    """
    批量处理数据
    
    Args:
        data: 数据列表
        num_items: 处理数量
        delay: 每次请求之间的延迟（秒），避免超过速率限制
    """
    results = []
    
    print(f"\n开始批量处理（共 {num_items} 条）")
    print("=" * 80)
    
    for i, item in enumerate(tqdm(data[:num_items], desc="处理进度")):
        print(f"\n[{i+1}/{num_items}] 处理 Video {item['video_id']}...")
        
        result = generate_description(item['video_id'], item['facts_text'])
        results.append(result)
        
        if result['status'] == 'success':
            print(f"  ✓ 成功")
        else:
            print(f"  ❌ 失败: {result['error']}")
        
        # 延迟，避免速率限制
        if i < num_items - 1:
            time.sleep(delay)
    
    # 统计
    success_count = sum(1 for r in results if r['status'] == 'success')
    fail_count = sum(1 for r in results if r['status'] == 'failed')
    
    print("\n" + "=" * 80)
    print(f"处理完成！")
    print(f"  成功: {success_count}/{len(results)}")
    print(f"  失败: {fail_count}/{len(results)}")
    
    return results

# 设置处理数量（可修改）
NUM_ITEMS = 10  # 先处理 10 条，测试效果

# 开始批量处理
results = process_batch(data, num_items=NUM_ITEMS, delay=1.0)

## 步骤 7: 查看结果

In [ ]:
# 显示前 2 条成功的结果
print("\n样本结果：")
print("=" * 80)

success_results = [r for r in results if r['status'] == 'success']

for i, result in enumerate(success_results[:2], 1):
    print(f"\n【结果 #{i}】")
    print(f"Video ID: {result['video_id']}")
    print(f"模型: {result.get('model', 'N/A')}")
    print(f"\n生成的描述:")
    print("-" * 80)
    print(result['description'])
    print("-" * 80)

## 步骤 8: 保存结果

In [ ]:
import json
from datetime import datetime

# 生成时间戳
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# 保存所有结果
output_file = f'/content/gemini_descriptions_{timestamp}.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"✓ 结果已保存: {output_file}")

# 保存失败记录
errors = [r for r in results if r['status'] == 'failed']
if errors:
    error_file = f'/content/errors_{timestamp}.json'
    with open(error_file, 'w', encoding='utf-8') as f:
        json.dump(errors, f, ensure_ascii=False, indent=2)
    print(f"✓ 错误记录已保存: {error_file}")

# 生成统计报告
report = {
    'total': len(results),
    'success': sum(1 for r in results if r['status'] == 'success'),
    'failed': sum(1 for r in results if r['status'] == 'failed'),
    'timestamp': timestamp
}

report_file = f'/content/report_{timestamp}.json'
with open(report_file, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2)

print(f"✓ 统计报告已保存: {report_file}")
print(f"\n📊 处理统计:")
print(f"  总数: {report['total']}")
print(f"  成功: {report['success']}")
print(f"  失败: {report['failed']}")

## 步骤 9: 下载结果

In [ ]:
from google.colab import files

# 下载结果文件
print("下载文件...")
files.download(output_file)

if errors:
    files.download(error_file)

files.download(report_file)

print("\n✓ 文件已下载到本地")

## 可选：处理更多数据

如果前面测试成功，可以处理更多数据：

In [ ]:
# 处理所有数据（100 条）
# ⚠️ 注意：这可能需要较长时间，并可能消耗 API 配额

# 取消下面的注释来运行
# all_results = process_batch(data, num_items=len(data), delay=1.5)

# # 保存
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# final_output = f'/content/gemini_all_descriptions_{timestamp}.json'
# with open(final_output, 'w', encoding='utf-8') as f:
#     json.dump(all_results, f, ensure_ascii=False, indent=2)

# print(f"✓ 全部结果已保存: {final_output}")
# files.download(final_output)

print("💡 取消注释上面的代码来处理全部数据")

## 📝 使用说明

### 运行顺序
1. 依次运行每个单元格
2. 在步骤 2 设置你的 Gemini API 密钥
3. 在步骤 3 上传或挂载数据文件
4. 步骤 5 会自动测试一条数据
5. 步骤 6 批量处理（默认 10 条）
6. 步骤 9 下载结果

### 注意事项
- ⚠️ 确保 API 密钥有效
- ⚠️ 注意 API 配额限制
- ⚠️ 批量处理时设置合理的延迟
- ⚠️ 定期保存结果，避免数据丢失

### 常见问题
1. **API 密钥无效**: 重新生成密钥
2. **速率限制**: 增加 `delay` 参数
3. **配额耗尽**: 等待重置或升级账户

### 结果文件
- `gemini_descriptions_*.json`: 所有结果
- `errors_*.json`: 失败记录
- `report_*.json`: 统计报告